# Labeling Algorithms

This notebook walks through segments of the PyMBE code base that pertain to generating labels from elements when there is not a clear user name but rather the label should be assembled from other attributes of the element and/or its owned elements.

The approach for complicated labels is to look at label fragments and how they can be combined in order to generate an integrated string.

In [1]:
import pymbe.api as pm
from pymbe.model import Element, Model

In [2]:
def label_expr(expression=None):
    
    if expression._metatype == 'OperatorExpression':
    # case for OperatorExpression - recurse on the parameters
    
        return f" {expression.operator} ".join(map(label_expr, expression.throughParameterMembership))
    
    # case for the Features under an expression
    elif expression._metatype == 'Feature':
        return label_expr(expression.throughFeatureValue[0])
    
    elif expression._metatype == 'FeatureReferenceExpression':
    # case for FeatureReferenceExpression - terminal case #1
        name_to_render = expression.throughMembership[0].declaredName
        return name_to_render
    elif expression._metatype == 'FeatureChainExpression':
        # first item will be FRE to another feature
        first_item = expression.throughParameterMembership[0].throughFeatureValue[0].throughMembership[0].declaredName
        # check if this is a two-item feature chain or n > 2
        if hasattr(expression, "throughMembership"):
            # this is the n = 2 case
            second_item = expression.throughMembership[0].declaredName
            return first_item + '.' + second_item
        else:
            # this is the n > 2 case
            chains = expression.throughOwningMembership[0].throughFeatureChaining
            other_items = '.'.join([chain.declaredName for chain in chains])
            return first_item + '.' + other_items
    elif hasattr(expression, "value"):
        return str(expression.value)
    
    elif expression._metatype == 'InvocationExpression':
        body = ', '.join(map(label_expr, expression.throughParameterMembership))
        return f"{expression.throughFeatureTyping[0].declaredName}({body})"
    else:
        raise NotImplementedError(f"Cannot process {expression._metatype} elements yet!")
    
    pass

In [3]:
new_model = Model.load_from_post_file("../../tests/fixtures/Model_Loader_Test_Level_3.json")

In [4]:
new_model

<SysML v2 Model (C:\Users\Bjorn Cole\Documents\GitHub\pyMBE\tests\fixtures\Model_Loader_Test_Level_3.json)>

In [5]:
new_model.ownedMetatype["Invariant"]

[<b1721dfe-65b3-44d9-a8e9-8de134a1b6c1 «Invariant»>,
 <bea8755b-7d58-4657-8c82-bd1a8616a9bf «Invariant»>,
 <ed65a371-da99-48a7-9abb-2ca6dbeb9da1 «Invariant»>,
 <beb7187e-22c3-4d0b-9a08-bb48bb54799f «Invariant»>,
 <d2e8e2ad-cc60-4f97-8f9d-a94d9abdb1cf «Invariant»>,
 <8e56739a-b360-4ca2-a5ab-2dfbb61e7506 «Invariant»>]

In [6]:
myInvar = new_model.ownedMetatype["Invariant"][-1]

In [7]:
myInvar

<8e56739a-b360-4ca2-a5ab-2dfbb61e7506 «Invariant»>

In [8]:
myInvar.throughResultExpressionMembership[0].operator

'=='

In [9]:
for invar in new_model.ownedMetatype["Invariant"]:
    print(label_expr(invar.throughResultExpressionMembership[0]))

Register 1 == Register 2 + Register 3
Register 2 == Register 3 + 15.0
Register 1 == Register 2 + Register 3
Register 1 == My Adder.Register 2 + My Adder.Register 3.Sub-Register 1.Sub-Sub-Register 2
sin(Register 1 + My Adder.Register 2) == 0.707
rect(Register 1, My Adder.Register 2) == 0.0


In [10]:
expr = myInvar.throughResultExpressionMembership[0]

In [11]:
expr.throughParameterMembership

[<195f2b6f-edd2-4f77-8f42-f16f6e5c0e06 «Feature»>,
 <83aa2197-f48f-434f-addc-62d236ca4eb8 «Feature»>]

In [12]:
expr.throughReturnParameterMembership

[<ef6f3768-3357-4074-a3ac-bf97289d6c7f «Feature»>]